# Development: Integrate PIBT Deadlock Resolution in RCA

In [73]:
import sys
sys.path.append('../')

sys.path.append('../../')
import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
from panav.environment.env import MultiTunnelEnv,DefaultEmtpyEnv
from panav.hybrid import HybridGraph

from panav.viz import draw_env

import shapely

from panav.ORCA import Ordered_Agent

%load_ext autoreload
%autoreload 2

%load_ext snakeviz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [111]:

bloating_r = 0.5
vmax = 1.0


results = []

env_name = "MultiTunnel"

# Current algorithm runs quickly under this config
# N = 20
# limits = [(-15,15),(-15,15)] 
# n_tunnel = 4

# Current algorithm runs slowly under this config
N = 40
limits = [(-15,15),(-20,20)] 
n_tunnel = 6

env = MultiTunnelEnv(n_tunnel = n_tunnel, tunnel_width = bloating_r*3.0,limits=limits, N_agent = N)
# env = DefaultEmtpyEnv(limits=limits, N_agent = N)

HG = HybridGraph(env,bloating_r)


/Users/tianpengzhang/opt/miniconda3/envs/PA-Nav/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


In [112]:
to_remove = []
for e in HG.edges:
    if HG.edges[e]['type']=='soft':
        # HG.edges[e]['weight'] = 0
        # pass
        u,v = e
        if HG.nodes[u]['type']== HG.nodes[v]['type']=='tunnel' and \
                                HG.nodes[u]['open_space_id'] == HG.nodes[v]['open_space_id']:
           to_remove.append(e) # Remove the soft edges connecting two tunnel endpoints for this particular environment
for e in to_remove:
    HG.remove_edge(*e)


In [113]:
from panav.TrafficAwarePlanning import traffic_aware_HG_plan
paths = traffic_aware_HG_plan(HG)

ref_plan = [np.array([HG.node_loc(u) for u in path]).T for path in paths]

In [114]:
plans = ref_plan

tau = 1.0 # The safe time interval. Can be generously long.
exec_tau = 0.8 * tau # Leaving a slight horizon margin helps avoid numerical inaccuracy in CVXPY optimization results.
agents = np.arange(N)
start_locs = env.starts
goal_locs = env.goals
# The execution time of ORCA velocity.
# Should be much shorter than the safe interval tau.

pos = []

v_prefs = [np.zeros(start_locs[0].shape) for a in agents]

protocol = 0

orcas = [Ordered_Agent(protocol,tau,bloating_r,vmax,p,init_v = None) 
         for p in start_locs]

# We will assume agent i is ranked i among the agents when dealing with conflicts.

curr_wp_index = [0 for a in agents]

def pwl_length(wps):
    '''
        The length of a piecewise linear path with waypoints=wps
    '''
    l = 0
    d,n = wps.shape
    for i in range(n-1):
        l += np.linalg.norm(wps[:,i]-wps[:,i+1])
    return l

def est_cost_to_go(agent):
    idx = curr_wp_index[agent]

    wps = np.hstack([orcas[agent].p.reshape(-1,1),ref_plan[agent]])

    return pwl_length(wps)


def calc_pref(agent):

    wp = plans[agent][:,curr_wp_index[agent]]
    # See if the agent has reached the current waypoint.
    if np.linalg.norm(wp-orcas[agent].p)<= bloating_r:  
        curr_wp_index[agent] = \
        np.min([curr_wp_index[agent]+1,   
                plans[agent].shape[1]-1
                                ])
        
        wp = plans[agent][:,curr_wp_index[agent]]
        
    to_wp = wp-orcas[agent].p
    
    v_prefs[agent] = to_wp/tau if tau * vmax > np.linalg.norm(to_wp) else vmax *  to_wp/(np.linalg.norm(to_wp)+1e-5)

def PIBT(a):

    P = []
    C = []

    for nb in agents:
        # Find all agents in the that could collide with a in the next tau seconds.
        if nb!=a and np.linalg.norm(orcas[nb].p-orcas[a].p)<orcas[nb].bloating_r+orcas[a].bloating_r\
                                                  + 3 * vmax * tau:
            if np.linalg.norm(orcas[nb].p-orcas[a].p)<orcas[nb].bloating_r+orcas[a].bloating_r:
                print("Soft Collision. Agents ",a,nb,"Dist",np.linalg.norm(orcas[nb].p-orcas[a].p))
                # raise Exception
            if orcas[nb].v is None:
                C.append(nb)
            else:
                P.append(nb)
            

    # print("Agent",a,"P",P,"C",C)

    candidate_v_pref = [] 

    for theta in np.pi * np.linspace(0,2,4)[:-1]:
            # Rotate v_pref clockwise by theta.
            v_right = np.array([[np.cos(-theta),-np.sin(-theta)],
                                [np.sin(-theta),np.cos(-theta)]]).dot(v_prefs[a])
            candidate_v_pref.append(v_right)

    candidate_v_pref.append(np.array([0,0])) # Always have zero velocity as a default 
    
    for i,v_pref in enumerate(candidate_v_pref):
        orcas[a].update_v(v_pref,env.obstacles,[orcas[b] for b in P]) 
        if orcas[a].v is None:
            # print("Agent",a,"infeasible")
            return False
        
        # print("Agent",a,"v_pref",i,"feasible",'v',orcas[a].v,'v_opt',orcas[a].v_opt)
        
        children_valid = True
        for c in C:
            if orcas[c].v is None:
                # print("Parent agent",a," calling PIBT for agent",c,"in recursive call")
                children_valid = PIBT(c)
                if not children_valid:
                    break
        
        if children_valid:
            return True

    orcas[a].v = None
    return False

for _ in range(400):
    print("################# Time step {} ################".format(_))
    pos.append(np.array([a.p for a in orcas]))
       
    for a in agents:
        # print("Agent",a)

        # Compute the preferred velocity.
        calc_pref(a)         

        prev_wp = curr_wp_index[a]-1
        cur_wp = curr_wp_index[a]
        u,v = paths[a][prev_wp:cur_wp+1]
        cur_edge_type = HG.edges[u,v]['type']
        if cur_edge_type == 'soft':
            orcas[a].vmax = 1.0 * vmax # On open space, move slower to avoid congestion at tunnel endpoints
        else:
            orcas[a].vmax = vmax # In tunnel, move quickly 
    

    # cost2go = [est_cost_to_go(a) for a in agents]
    # agents = agents[np.argsort(cost2go)] # Priority agents that are closer to their goals
    
    for a in agents: # Priority agents that are closer to their goals
        if orcas[a].v is None:
            # print("PIBT for agent",a,"in outer loop")
            valid = PIBT(a)
            # print('Agent',a,'Valid',valid," in outer loop")
    
    # We assume agent a is ranked a among the agents when dealing with conflicts.
    
    # Execute the safe velocity.
    all_reached = True
    
    for a in agents:
        dist2goal = np.linalg.norm(orcas[a].p - goal_locs[a])
        # print(a,dist2goal)

        if dist2goal>=1*bloating_r:
            orcas[a].p += orcas[a].v*exec_tau
            all_reached = False
            # Reset all agent's v to be None
            orcas[a].v = None
        else:
            orcas[a].v = orcas[a].v_opt = np.array([0,0])
    if all_reached:
        break

################# Time step 0 ################
################# Time step 1 ################
################# Time step 2 ################
################# Time step 3 ################
################# Time step 4 ################
################# Time step 5 ################
################# Time step 6 ################
################# Time step 7 ################
################# Time step 8 ################
################# Time step 9 ################
################# Time step 10 ################
################# Time step 11 ################
################# Time step 12 ################
################# Time step 13 ################
################# Time step 14 ################
################# Time step 15 ################
################# Time step 16 ################
################# Time step 17 ################
################# Time step 18 ################
################# Time step 19 ################
################# Time step 20 ################
##

In [115]:
from panav.viz import animation
from IPython.display import HTML, display
pos = np.array(pos)

dt = exec_tau * 0.25

pos_to_ani = [pos[:,a,:].T for a in agents]
anim = animation(env, pos_to_ani, bloating_r, dt = dt)
display(HTML(anim.to_jshtml()))
plt.close() 